In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 79.4 MB/s 


In [ ]:
import pickle
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import json

In [ ]:
loaded_model = load_model('/content/universe/hyol_model3.h5')

In [ ]:
okt = Okt()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다','로','것','고','원']
max_len = 100

In [ ]:
with open('/content/universe/tokenizer.pickle','rb') as handle:
  tokenizer = pickle.load(handle)

In [ ]:
def get_word_and_token(sentense):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', sentense)
  word_array = okt.morphs(new_sentence, stem=True) 
  word_array = [word for word in word_array if not word in stopwords] 
  token_array = tokenizer.texts_to_sequences([word_array])
  return word_array, token_array[0]

In [ ]:
def write_new_json(input_json_path, output_json_path):
  with open(input_json_path, 'r') as input_json_file:
    place_data = json.load(input_json_file)

    # delete non-review place
    place_data = list(filter(lambda place: len(place["placeReviews"]) > 0, place_data))

    # update review to [review, token]
    for place in place_data:
      place["placeWordCount"] = {}
      temp_reviews = place["placeReviews"].copy()
      temp_review_starts = place["placeReviewStar"].copy()
      for idx, review in enumerate(place["placeReviews"]):
        if len(review) > 0:
          word_array, token_array = get_word_and_token(review)
          temp_reviews[idx] = {
              "review": review,
              "token": token_array
          }
          for word in word_array:
            try:
              place["placeWordCount"][word] += 1
            except:
              place["placeWordCount"][word] = 1

      place["placeWordCount"] = dict(sorted(place["placeWordCount"].items(), key=lambda x:x[1], reverse = True))

      # delete empty review
      place["placeReviews"] = list(filter(lambda review: len(review) > 0, temp_reviews))

      del(place["placeTotalReview"])
      del(place["placeReviewStar"])

    with open(output_json_path, 'w', encoding="UTF-8") as output_json_file:
      json.dump(place_data, output_json_file, indent=4, ensure_ascii=False)
      
      

In [ ]:
get_word_and_token("더럽고 최악. 불편함. 불편함 그리고 직원들 싸가지없음")

['더럽다', '최악', '불편하다', '불편하다', '그리고', '직원', '싸가지', '없다']
[[417, 120, 199, 199, 107, 12, 337, 4]]


In [ ]:
if __name__ == "__main__":
  print("맛집 start")
  write_new_json("/content/universe/output_맛집.json", "/content/universe/output_맛집_new.json")
  print("맛집 end")
  print("치킨 start")
  write_new_json("/content/universe/output_치킨.json", "/content/universe/output_치킨_new.json")
  print("치킨 end")
  print("카페 start")
  write_new_json("/content/universe/output_카페.json", "/content/universe/output_카페_new.json")
  print("카페 end")

맛집 start
맛집 end
치킨 start
치킨 end
카페 start
카페 end
